### FLIGHT CANCELLATION AND DELAY ANALYSIS 

In [22]:
#Importing dependencies 
import pandas as pd 
import datetime
import matplotlib.pyplot as plt
import sklearn


In [23]:
#Importing dataset 

flights_df = pd.read_csv('flight_data.csv')
flights_df.head(5)

,AIRLINE,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,...,dest_SNWD,dest_AWND,dest_PRCP,CRS_DEP_Datetime,15_DELAYED,year,month,day,day_of_week,hour
0,Spirit Air Lines,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",-2.0,2041,-1.0,0.0,0.0,...,0.0,55.0,8.0,2020-02-23 18:40:00,0,2020,2,23,6,18
1,Delta Air Lines Inc.,ATL,"Atlanta, GA",BDL,"Hartford, CT",69.0,2112,60.0,0.0,0.0,...,0.0,30.0,0.0,2021-06-11 18:50:00,1,2021,6,11,4,18
2,American Airlines Inc.,RDU,"Raleigh/Durham, NC",DFW,"Dallas/Fort Worth, TX",-6.0,1358,-1.0,0.0,0.0,...,0.0,45.0,0.0,2020-07-31 12:07:00,0,2020,7,31,4,12
3,Delta Air Lines Inc.,BDL,"Hartford, CT",ATL,"Atlanta, GA",-5.0,840,-32.0,0.0,0.0,...,0.0,36.0,0.0,2019-11-20 06:00:00,0,2019,11,20,2,6
4,Southwest Airlines Co.,BWI,"Baltimore, MD",BDL,"Hartford, CT",3.0,1840,6.0,0.0,0.0,...,0.0,26.0,0.0,2022-05-01 17:35:00,0,2022,5,1,6,17


In [24]:
#Checking numnber of rows 
num_rows = flights_df.shape[0]
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 1255864


In [25]:
#METADATA FUNCTION 

def get_metadata(df) :
    #  Sanity Check (raise TypeError):
    if not isinstance(df, pd.DataFrame) :
        raise TypeError("ERROR: Not a Pandas DataFrame object")

    # Column (feature) names:
    col_names = df.columns

    # Column (feature) types:
    col_dtypes = df.dtypes
    dtypes_list = []
    for i in range(0, len(col_dtypes)) :
        dtypes_list.append(col_dtypes.iloc[i])

    # Missing Values by Column:
    col_missing = df.isna().sum()

    # Metadata DataFrame:
    metadata_df = pd.DataFrame({
        "column_names": col_names,
        "column_dtypes": dtypes_list,
        "column_missing": col_missing,
    }).reset_index(drop=True)

    return metadata_df

In [26]:
#EXTRACTING METADATA 
metadata_df = get_metadata(df=flights_df)
metadata_df = metadata_df.sort_values(by=["column_missing"], ascending=False)
#metadata_df.to_csv("metadata.csv")
metadata_df

,column_names,column_dtypes,column_missing
7,ARR_DELAY,float64,36062
5,DEP_DELAY,float64,32515
0,AIRLINE,object,0
16,ori_PRCP,float64,0
28,day_of_week,int64,0
27,day,int64,0
26,month,int64,0
25,year,int64,0
24,15_DELAYED,int64,0
23,CRS_DEP_Datetime,object,0


In [27]:
#Delayed flights based on the arrival time, delayed more than 15 mins 
arr_delayed_flights = (flights_df['15_DELAYED']== 1).sum()
print("Number of delayed flights in df:", arr_delayed_flights)

Number of delayed flights in df: 275819


In [28]:
#Checking how many flights eacha airline operates 
 
flights_per_airline = flights_df['AIRLINE'].value_counts()
print(flights_per_airline)

AIRLINE
Delta Air Lines Inc.                  205787
Southwest Airlines Co.                179311
American Airlines Inc.                169145
United Air Lines Inc.                 131047
SkyWest Airlines Inc.                 115824
Endeavor Air Inc.                      64195
Envoy Air                              61379
Republic Airline                       59153
PSA Airlines Inc.                      46596
Spirit Air Lines                       45785
JetBlue Airways                        37839
Frontier Airlines Inc.                 37746
Mesa Airlines Inc.                     33803
Alaska Airlines Inc.                   29299
Allegiant Air                          21549
ExpressJet Airlines LLC d/b/a aha!     10858
Horizon Air                             6548
Name: count, dtype: int64


### DATA CLEANING & FEATURE ENGINEERING 

In [29]:
#Creating a is weekend variable 

flights_df['is_weekend'] = (flights_df['day_of_week'] >= 4).astype(int) #1 means weekend 
flights_df.head(5)

,AIRLINE,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,...,dest_AWND,dest_PRCP,CRS_DEP_Datetime,15_DELAYED,year,month,day,day_of_week,hour,is_weekend
0,Spirit Air Lines,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",-2.0,2041,-1.0,0.0,0.0,...,55.0,8.0,2020-02-23 18:40:00,0,2020,2,23,6,18,1
1,Delta Air Lines Inc.,ATL,"Atlanta, GA",BDL,"Hartford, CT",69.0,2112,60.0,0.0,0.0,...,30.0,0.0,2021-06-11 18:50:00,1,2021,6,11,4,18,1
2,American Airlines Inc.,RDU,"Raleigh/Durham, NC",DFW,"Dallas/Fort Worth, TX",-6.0,1358,-1.0,0.0,0.0,...,45.0,0.0,2020-07-31 12:07:00,0,2020,7,31,4,12,1
3,Delta Air Lines Inc.,BDL,"Hartford, CT",ATL,"Atlanta, GA",-5.0,840,-32.0,0.0,0.0,...,36.0,0.0,2019-11-20 06:00:00,0,2019,11,20,2,6,0
4,Southwest Airlines Co.,BWI,"Baltimore, MD",BDL,"Hartford, CT",3.0,1840,6.0,0.0,0.0,...,26.0,0.0,2022-05-01 17:35:00,0,2022,5,1,6,17,1


In [30]:
#Creating a is weekday variable 
flights_df['is_weekday'] = (flights_df['day_of_week'] < 4).astype(int) #1 means weekday 
flights_df.head(5)

,AIRLINE,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,...,dest_PRCP,CRS_DEP_Datetime,15_DELAYED,year,month,day,day_of_week,hour,is_weekend,is_weekday
0,Spirit Air Lines,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",-2.0,2041,-1.0,0.0,0.0,...,8.0,2020-02-23 18:40:00,0,2020,2,23,6,18,1,0
1,Delta Air Lines Inc.,ATL,"Atlanta, GA",BDL,"Hartford, CT",69.0,2112,60.0,0.0,0.0,...,0.0,2021-06-11 18:50:00,1,2021,6,11,4,18,1,0
2,American Airlines Inc.,RDU,"Raleigh/Durham, NC",DFW,"Dallas/Fort Worth, TX",-6.0,1358,-1.0,0.0,0.0,...,0.0,2020-07-31 12:07:00,0,2020,7,31,4,12,1,0
3,Delta Air Lines Inc.,BDL,"Hartford, CT",ATL,"Atlanta, GA",-5.0,840,-32.0,0.0,0.0,...,0.0,2019-11-20 06:00:00,0,2019,11,20,2,6,0,1
4,Southwest Airlines Co.,BWI,"Baltimore, MD",BDL,"Hartford, CT",3.0,1840,6.0,0.0,0.0,...,0.0,2022-05-01 17:35:00,0,2022,5,1,6,17,1,0


In [31]:
#Feature selection / dropping columns 

#columns_to_drop = ["ARR_DELAY", "DEP_DELAY", "CANCELLED", "DIVERTED", "CRS_DEP_Datetime", "CRS_ARR_TIME"]

new_flights = flights_df.drop(
    columns = ["ARR_DELAY", "DEP_DELAY", "ORIGIN_CITY","DEST_CITY" ,"CANCELLED", "DIVERTED", "CRS_DEP_Datetime", "CRS_ARR_TIME"]
)
new_flights

,AIRLINE,ORIGIN,DEST,DISTANCE,ori_TMIN,ori_TMAX,ori_SNOW,ori_SNWD,ori_AWND,ori_PRCP,...,dest_AWND,dest_PRCP,15_DELAYED,year,month,day,day_of_week,hour,is_weekend,is_weekday
0,Spirit Air Lines,MCO,DFW,985.0,117.0,256.0,0.0,0.0,32.0,0.0,...,55.0,8.0,0,2020,2,23,6,18,1,0
1,Delta Air Lines Inc.,ATL,BDL,859.0,222.0,294.0,0.0,0.0,35.0,122.0,...,30.0,0.0,1,2021,6,11,4,18,1,0
2,American Airlines Inc.,RDU,DFW,1061.0,250.0,333.0,0.0,0.0,40.0,0.0,...,45.0,0.0,0,2020,7,31,4,12,1,0
3,Delta Air Lines Inc.,BDL,ATL,859.0,39.0,67.0,0.0,0.0,35.0,3.0,...,36.0,0.0,0,2019,11,20,2,6,0,1
4,Southwest Airlines Co.,BWI,BDL,283.0,67.0,189.0,0.0,0.0,20.0,56.0,...,26.0,0.0,0,2022,5,1,6,17,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255859,Spirit Air Lines,DFW,LAS,1055.0,267.0,367.0,0.0,0.0,25.0,0.0,...,43.0,0.0,0,2021,8,26,3,6,0,1
1255860,Delta Air Lines Inc.,ATL,EWR,746.0,6.0,167.0,0.0,0.0,30.0,0.0,...,28.0,0.0,0,2022,2,9,2,8,0,1
1255861,American Airlines Inc.,JAX,CLT,328.0,89.0,194.0,0.0,0.0,34.0,0.0,...,50.0,0.0,0,2022,11,13,6,17,1,0
1255862,American Airlines Inc.,ORD,AUS,977.0,61.0,222.0,0.0,0.0,26.0,0.0,...,18.0,0.0,0,2022,11,2,2,13,0,1


### CREATING TRAINING AND TESTING SETS 

In [32]:
# Define a function to replace infrequent categories with 'Other'
def replace_infrequent_categories(df, categorical_cols, threshold=0.01, other_label='Other'):
    for col in categorical_cols:
        counts = df[col].value_counts(normalize=True)
        infrequent_categories = counts[counts < threshold].index
        df[col] = df[col].replace(infrequent_categories, other_label)
    return df

# Define threshold for infrequent categories (adjust as needed)
threshold = 0.01
print("w")


w


In [33]:
from sklearn.model_selection import train_test_split

X = new_flights.drop(columns=['15_DELAYED'])
y= new_flights["15_DELAYED"]

#new_flights_df = replace_infrequent_categories(X,)

#X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=0)

### CREATING PIPELINES FOR CATEGORICAL AND NUMERICAL VALUES 

In [34]:
X.columns

Index(['AIRLINE', 'ORIGIN', 'DEST', 'DISTANCE', 'ori_TMIN', 'ori_TMAX',
       'ori_SNOW', 'ori_SNWD', 'ori_AWND', 'ori_PRCP', 'dest_TMIN',
       'dest_TMAX', 'dest_SNOW', 'dest_SNWD', 'dest_AWND', 'dest_PRCP', 'year',
       'month', 'day', 'day_of_week', 'hour', 'is_weekend', 'is_weekday'],
      dtype='object')

In [35]:
cat_features = X.select_dtypes(include=["object"])

cat_features = cat_features.assign(
    AIRLINE=X["AIRLINE"],
    ORIGIN=X["ORIGIN"],
    DEST=X["DEST"],
    is_weekend=X["is_weekend"].astype(str),
    is_weekday=X["is_weekday"].astype(str),

)

new_flights_df = replace_infrequent_categories(X,cat_features, threshold)

X= new_flights_df

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=0)

# Categorical Pipeline:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

cat_pipe = Pipeline(
    # This pipeline is designed to preprocess categorical features.
    steps=[
        ("one-hot-encode", OneHotEncoder(handle_unknown="ignore"))
    ]
)

encoded_cat_features = cat_pipe.fit_transform(cat_features)
encoded_cat_features

<1255864x462 sparse matrix of type '<class 'numpy.float64'>'
	with 6279320 stored elements in Compressed Sparse Row format>

### NUMERIC PIPELINE

In [36]:
from sklearn.preprocessing import StandardScaler

numeric_features = X[X.columns[~ X.columns.isin(cat_features.columns.tolist())]]



num_pipe = Pipeline(
    #This pipeline is designed to preprocess numeric features.
    steps=[
        ("standardization", StandardScaler()),
    ]
)

num_pipe

Pipeline(steps=[('standardization', StandardScaler())])

### PUTTING THE PIPELINES TOGETHER

In [37]:
from sklearn.compose import ColumnTransformer

preprocessing_pipe = ColumnTransformer(
    [
        ("numeric-pipe", num_pipe, numeric_features.columns.tolist()),
        ("categorical-pipe", cat_pipe, cat_features.columns.tolist()),
    ]
)

preprocessing_pipe

ColumnTransformer(transformers=[('numeric-pipe',
                                 Pipeline(steps=[('standardization',
                                                  StandardScaler())]),
                                 ['DISTANCE', 'ori_TMIN', 'ori_TMAX',
                                  'ori_SNOW', 'ori_SNWD', 'ori_AWND',
                                  'ori_PRCP', 'dest_TMIN', 'dest_TMAX',
                                  'dest_SNOW', 'dest_SNWD', 'dest_AWND',
                                  'dest_PRCP', 'year', 'month', 'day',
                                  'day_of_week', 'hour']),
                                ('categorical-pipe',
                                 Pipeline(steps=[('one-hot-encode',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['AIRLINE', 'ORIGIN', 'DEST', 'is_weekend',
                                  'is_weekday'])])

### FITTING THE DECISION TREE

In [38]:
from sklearn.tree import DecisionTreeClassifier as dt


model = Pipeline(
    steps=[
        ("preprocessing-pipe", preprocessing_pipe),
        ("decision-tree-model", dt(class_weight='balanced'))
    ]
)


model

Pipeline(steps=[('preprocessing-pipe',
                 ColumnTransformer(transformers=[('numeric-pipe',
                                                  Pipeline(steps=[('standardization',
                                                                   StandardScaler())]),
                                                  ['DISTANCE', 'ori_TMIN',
                                                   'ori_TMAX', 'ori_SNOW',
                                                   'ori_SNWD', 'ori_AWND',
                                                   'ori_PRCP', 'dest_TMIN',
                                                   'dest_TMAX', 'dest_SNOW',
                                                   'dest_SNWD', 'dest_AWND',
                                                   'dest_PRCP', 'year', 'month',
                                                   'day', 'day_of_week',
                                                   'hour']),
                                                 ('categorical-pipe',
                                                  Pipeline(steps=[('one-hot-encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['AIRLINE', 'ORIGIN', 'DEST',
                                                   'is_weekend',
                                                   'is_weekday'])])),
                ('decision-tree-model',
                 DecisionTreeClassifier(class_weight='balanced'))])

### TRAIN AND EVALUATE THE MODEL 

In [39]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing-pipe',
                 ColumnTransformer(transformers=[('numeric-pipe',
                                                  Pipeline(steps=[('standardization',
                                                                   StandardScaler())]),
                                                  ['DISTANCE', 'ori_TMIN',
                                                   'ori_TMAX', 'ori_SNOW',
                                                   'ori_SNWD', 'ori_AWND',
                                                   'ori_PRCP', 'dest_TMIN',
                                                   'dest_TMAX', 'dest_SNOW',
                                                   'dest_SNWD', 'dest_AWND',
                                                   'dest_PRCP', 'year', 'month',
                                                   'day', 'day_of_week',
                                                   'hour']),
                                                 ('categorical-pipe',
                                                  Pipeline(steps=[('one-hot-encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['AIRLINE', 'ORIGIN', 'DEST',
                                                   'is_weekend',
                                                   'is_weekday'])])),
                ('decision-tree-model',
                 DecisionTreeClassifier(class_weight='balanced'))])

In [40]:
from sklearn.metrics import accuracy_score ,  precision_score, recall_score,mean_squared_error as MSE, r2_score
import numpy as np

# Get predictions from testing set
y_pred = model.predict(X_test)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average= None)
recall = recall_score(y_test, y_pred, average= None)

print(f"RMSE is: {np.sqrt(MSE(y_test, y_pred))}")
print(f"\nR^2 is: {r2_score(y_test, y_pred) * 100} %")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

RMSE is: 0.5401804312792188

R^2 is: -69.98636721497526 %
Accuracy: 0.7082051016629972
Precision: [0.81547337 0.34166886]
Recall: [0.80889177 0.35143984]


In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.81      0.81    195889
           1       0.34      0.35      0.35     55284

    accuracy                           0.71    251173
   macro avg       0.58      0.58      0.58    251173
weighted avg       0.71      0.71      0.71    251173



### HYPERPARAMETER TUNING 


In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'decision-tree-model__max_depth':[None, 5, 10,15], 
    'decision-tree-model__min_samples_split':[2,5,10], 
    'decision-tree-model__min_samples_leaf':[1,2,4], 
    'decision-tree-model__max_features':['auto', 'sqrt','log2']
}

grid_search= GridSearchCV(estimator=model, param_grid=param_grid, scoring='recall', cv=5)

grid_search.fit(X_train, y_train)



/Users/fernandacarrillo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/fernandacarrillo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/fernandacarrillo/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/fernandacarrillo/anaconda3/lib/python3.11

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing-pipe',
                                        ColumnTransformer(transformers=[('numeric-pipe',
                                                                         Pipeline(steps=[('standardization',
                                                                                          StandardScaler())]),
                                                                         ['DISTANCE',
                                                                          'ori_TMIN',
                                                                          'ori_TMAX',
                                                                          'ori_SNOW',
                                                                          'ori_SNWD',
                                                                          'ori_AWND',
                                                                          'ori_PRCP',
                                                                          'dest_TMIN',
                                                                          'dest_TMAX',
                                                                          'dest_SNOW',
                                                                          'dest_SNWD',
                                                                          'dest_AWND',
                                                                          'dest_PRCP',
                                                                          'year',
                                                                          'month',
                                                                          'day',
                                                                          'day_of_we...
                                                                          'ORIGIN',
                                                                          'DEST',
                                                                          'is_weekend',
                                                                          'is_weekday'])])),
                                       ('decision-tree-model',
                                        DecisionTreeClassifier(class_weight='balanced'))]),
             param_grid={'decision-tree-model__max_depth': [None, 5, 10, 15],
                         'decision-tree-model__max_features': ['auto', 'sqrt',
                                                               'log2'],
                         'decision-tree-model__min_samples_leaf': [1, 2, 4],
                         'decision-tree-model__min_samples_split': [2, 5, 10]},
             scoring='recall')

In [20]:
#Exploring best hyperparameters

grid_search.best_params_

{'decision-tree-model__max_depth': 5,
 'decision-tree-model__max_features': 'log2',
 'decision-tree-model__min_samples_leaf': 4,
 'decision-tree-model__min_samples_split': 2}

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Get the best estimator from grid search
best_estimator = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_estimator.predict(X_test)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)


Accuracy: 0.5723823818642927
Precision: 0.2784531420021763
Recall: 0.5924860719195427


In [ ]:
feature_importances = grid_search.feature_importances

In [22]:
## Extract all Cross-Validation results:
cv_results= grid_search.cv_results_

## Store the results in a Pandas DataFrame:
cv_results_df= pd.DataFrame(
    cv_results
)
## 100-Total Elastic Nets Made:
cv_results_df 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_decision-tree-model__max_depth,param_decision-tree-model__max_features,param_decision-tree-model__min_samples_leaf,param_decision-tree-model__min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.216077,0.220699,0.000000,0.000000,None,auto,1,2,"{'decision-tree-model__max_depth': None, 'deci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,73
1,2.123128,0.166586,0.000000,0.000000,None,auto,1,5,"{'decision-tree-model__max_depth': None, 'deci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,73
2,2.620723,0.405399,0.000000,0.000000,None,auto,1,10,"{'decision-tree-model__max_depth': None, 'deci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,73
3,2.354197,0.102723,0.000000,0.000000,None,auto,2,2,"{'decision-tree-model__max_depth': None, 'deci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,73
4,2.273990,0.099074,0.000000,0.000000,None,auto,2,5,"{'decision-tree-model__max_depth': None, 'deci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,17.849094,29.052981,0.553445,0.079803,15,log2,2,5,"{'decision-tree-model__max_depth': 15, 'decisi...",0.780336,0.780096,0.780251,0.780290,0.780280,0.780251,0.000082,42
104,9.696663,10.796123,19.843529,38.546979,15,log2,2,10,"{'decision-tree-model__max_depth': 15, 'decisi...",0.780411,0.780430,0.780435,0.780395,0.780236,0.780381,0.000074,36
105,6.544560,4.143799,0.512530,0.022316,15,log2,4,2,"{'decision-tree-model__max_depth': 15, 'decisi...",0.780237,0.780330,0.780365,0.780405,0.780360,0.780339,0.000057,39
106,21.879528,35.840608,182.143306,363.251409,15,log2,4,5,"{'decision-tree-model__max_depth': 15, 'decisi...",0.780371,0.780266,0.780475,0.780196,0.780295,0.780321,0.000095,40
